<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp3_parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from google.colab import drive 
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import functools

drive.mount('/content/gdrive')
path_a_training_set = 'gdrive/MyDrive/TP3 dataset music/train.parquet'
path_a_test_set = 'gdrive/MyDrive/TP3 dataset music/test.parquet'


df_music_train = pd.read_parquet(path_a_training_set).fillna("")
df_music_test = pd.read_parquet(path_a_test_set).fillna("")

Mounted at /content/gdrive


## Preprocessing

In [2]:
nltk.download('vader_lexicon')
def apply_sentiment_analysis_to_lyrics(df_music: pd.DataFrame) -> pd.DataFrame:
  sia = SentimentIntensityAnalyzer()
  negative, neutral, positive, compound = 0, 1, 2, 3
  sentimentAnalysisOfLyrics = df_music["lyric"].map(lambda lyric: list(sia.polarity_scores(lyric).values()))
  negativeScoreOfLyrics = sentimentAnalysisOfLyrics.map(lambda row: row[negative])
  positiveScoreOfLyrics = sentimentAnalysisOfLyrics.map(lambda row: row[positive])
  neutralScoreOfLyrics = sentimentAnalysisOfLyrics.map(lambda row: row[neutral])
  compoundScoreOfLyrics = sentimentAnalysisOfLyrics.map(lambda row: row[compound])
  return pd.DataFrame(pd.concat([negativeScoreOfLyrics, positiveScoreOfLyrics,neutralScoreOfLyrics,compoundScoreOfLyrics], axis = 1))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
nltk.download('stopwords')
def get_sum_tfidf_from_lyrics(df_music: pd.DataFrame) -> pd.DataFrame:
  stopwords = set(nltk.corpus.stopwords.words("english")).union(set(nltk.corpus.stopwords.words("spanish"))).union(set(nltk.corpus.stopwords.words("french")))
  vectorizer = TfidfVectorizer(input = "content", stop_words = stopwords)
  vectorizer.fit(df_music["lyric"])
  return pd.DataFrame([np.sum(tfidf_vector) for tfidf_vector in vectorizer.transform(df_music["lyric"])])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def eliminate_genres_without_enough_observations(df_music: pd.DataFrame) -> pd.DataFrame:
  df_music_recuento_filas_por_genero = df_music.groupby("genre").count().reset_index()[["genre", "track_name"]].rename(
    columns = {"track_name": "rowCount"}).sort_values("rowCount")
  problematic_genres = list(df_music_recuento_filas_por_genero[df_music_recuento_filas_por_genero["rowCount"] < 50].genre)[1:]
  return df_music[~df_music["genre"].isin(problematic_genres)]

In [5]:
nltk.download('punkt')
def get_length_transforms_for_text(df_music: pd.DataFrame) -> pd.DataFrame:
  df_music["number_of_lines"] = df_music["lyric"].map(lambda lyric: len(lyric.split("\n")))
  df_music["number_of_tokens"] = df_music["lyric"].map(lambda lyric: len(nltk.word_tokenize(lyric)))
  df_music["length_lyrics"] = df_music["lyric"].map(lambda lyric: len(lyric))
  df_music["length_of_track_name"] = df_music["track_name"].map(lambda track_name: len(track_name))
  return df_music[["length_lyrics", "length_of_track_name", "number_of_lines", "number_of_tokens"]]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
nltk.download('averaged_perceptron_tagger')

def get_POS_vector_from_lyric_POS(lyric_POS):
  useful_pos_tags = ["FW", "JJR", "NN", "NNS", "NNP", "PDT", "PRP", "RB", "RBR",
                   "UH", "VB", "VBG", "VBD", "VBN", "VBP", "VBZ"]
  POS_dictionary = {pos_tag:0 for pos_tag in useful_pos_tags}
  POS_dictionary.update({"other":0})

  for token in lyric_POS:
    if token[1] in useful_pos_tags:
      POS_dictionary[token[1]] += 1
    else:
      POS_dictionary["other"] += 1

  return [POS_dictionary[pos_tag] for pos_tag in POS_dictionary]
  

def get_POS_chunk_taggin_counts_for_text(df_music: pd.DataFrame) -> pd.DataFrame:
  df_music["lyric_POS"] = df_music["lyric"].map(lambda lyric: nltk.pos_tag(nltk.word_tokenize(lyric)))
  df_music["lyric_POS"] = df_music["lyric_POS"].map(lambda lyric_pos: get_POS_vector_from_lyric_POS(lyric_pos))
  return pd.DataFrame(df_music["lyric_POS"].to_list())


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
df_music_filtered_genres = eliminate_genres_without_enough_observations(df_music_train)
artists = list(df_music_filtered_genres["artist"].unique())
train_artists = set(artists[:int(0.9*len(artists))])
validation_artists = set(artists[int(0.9*len(artists)):])
train_set = df_music_filtered_genres[df_music_filtered_genres["artist"].isin(train_artists)]
validation_set = df_music_filtered_genres[df_music_filtered_genres["artist"].isin(validation_artists)]

In [8]:
def mean_hot_encoder_for_lang(df_music: pd.DataFrame, df_training: pd.DataFrame) -> pd.DataFrame:
  df_training_grouped_by_lang = df_training.groupby("language").mean().reset_index()[["language", "popularity", "a_popularity", "loudness"]]
  df_new_columns = df_music.merge(df_training_grouped_by_lang, on = "language", how = "left")
  return df_new_columns[["popularity_y", "a_popularity_y", "loudness_y"]].fillna(0)

mean_hot_encoder_using_training_set = lambda df_to_encode: mean_hot_encoder_for_lang(df_to_encode, df_music_train) 

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

text_features = ["track_name", "lyric", "artist"]

numerical_features = ["a_songs", "a_popularity", "popularity", "acousticness", "danceability", "duration_ms",
                   "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"]

ordinal_features = ["key", "time_signature"]

one_hot_features = ["mode"]

mean_enc_features = ["language"]

artist_genres = ["a_genres", "genre"]

label = ["genre"]

identity_transformer = preprocessing.FunctionTransformer(None)

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df_music_train.genre)

full_processor = ColumnTransformer(transformers=[
    ('text_sentiment_analysis', preprocessing.FunctionTransformer(apply_sentiment_analysis_to_lyrics), text_features),
    ('text_tf_idf', preprocessing.FunctionTransformer(get_sum_tfidf_from_lyrics), text_features),
    ('text_simple_transforms', preprocessing.FunctionTransformer(get_length_transforms_for_text), text_features),
    ('text_POS_count', preprocessing.FunctionTransformer(get_POS_chunk_taggin_counts_for_text), text_features),
    ('mean_encoding', preprocessing.FunctionTransformer(mean_hot_encoder_using_training_set), list(df_music_train.columns)),
    ('one_hot_encoding', preprocessing.OneHotEncoder(), one_hot_features),
    ('numerical', identity_transformer, numerical_features),
    ('ordinal', preprocessing.OrdinalEncoder(categories = [['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'],
                                                            ['1/4', '3/4', '4/4', '5/4']]), ordinal_features)])


## XGBoost

In [14]:
import xgboost as xgb
import multiprocessing

In [19]:
X_train = full_processor.fit_transform(train_set)
y_train = label_encoder.transform(train_set.genre)
X_validation = full_processor.transform(validation_set)
y_validation = label_encoder.transform(validation_set.genre)

In [42]:
df_X = pd.DataFrame(X_train)
df_X.columns = ["negativeScore", "positiveScore", "neutralScore", "compoundScore"] + ["sumTfidf"] + \
                  ["length_lyrics", "length_of_track_name", "number_of_lines", "number_of_tokens"] + \
                  ["FW", "JJR", "NN", "NNS", "NNP", "PDT", "PRP", "RB", "RBR",
                   "UH", "VB", "VBG", "VBD", "VBN", "VBP", "VBZ", "other"] + \
                  ["Minus", "Major"] + \
                  ["popularity_by_lang", "a_popularity_by_lang", "loudness_by_lang"] + numerical_features + ordinal_features

df_X[ordinal_features] = df_X[ordinal_features].astype("category")

In [26]:
modelo_xgb = xgb.XGBClassifier(enable_categorical = True, booster = "gbtree")
modelo_xgb.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [35]:
preds = modelo_xgb.predict_proba(X_validation)
top_k_accuracy_score(y_validation, preds, labels = modelo_xgb.classes_)

0.5281536446966391

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import poisson

search_space = {
    "learning_rate" : expon(1/5),
    "gamma" : expon(),
    "max_depth" : poisson(mu = 10),
    "min_child_weight": poisson(mu = 3),
    "reg_lambda" : expon(),
    "reg_alpha" : expon(),
}

search = RandomizedSearchCV(xgb.XGBClassifier(booster = "gbtree", tree_method = "exact"), param_distributions = search_space, cv = 3, n_iter = 20, 
                            scoring = "accuracy")

In [64]:
optimized_XGBoost = search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


In [65]:
optimized_XGBoost.best_params_

{'gamma': 0.030640848396389538,
 'learning_rate': 0.36870060959122297,
 'max_depth': 12,
 'min_child_weight': 4,
 'reg_alpha': 1.2000833745864283,
 'reg_lambda': 1.556162649431694}

In [70]:
top_k_accuracy_score(y_validation, optimized_XGBoost.predict_proba(X_validation), labels = optimized_XGBoost.classes_)

0.5189873417721519

In [74]:
modelo_xgb = xgb.XGBClassifier(enable_categorical = True, booster = "gbtree", learning_rate = 0.3,
                              max_depth = 26, min_child_weight = 4,
                               reg_alpha = 1.2000833745864283, reg_lambda =  1.556162649431694)

modelo_xgb.fit(X_train, y_train)
preds = modelo_xgb.predict_proba(X_validation)
top_k_accuracy_score(y_validation, preds, labels = modelo_xgb.classes_)

KeyboardInterrupt: ignored